## Fully Connected layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

# input dimensions
t_max = 2**7
dims = (1,1,5,5)
n_outputs = 10
np.random.seed(seed=31)

# model parameters
weights = (np.random.rand(n_outputs,np.product(dims[1:])) - 0.5) / 2 # 
# weights = np.ones((n_outputs,np.product(dims[1:])))  # 
# weights = np.array([[0.5],[-0.5]])
biases = (np.random.rand(n_outputs) - 0.5) / 2 # 
# biases = np.zeros((n_outputs)) / -4 # np.ones((n_outputs)) / 2 # 
inputs = np.random.rand(*dims) / 2 # np.ones((dims)) * 0.5 #  
# inputs = np.zeros((dims)) / 4 # 
#inputs = np.array([0.5, 0.25])

loihi_model = quartz.Network(t_max, [
    layers.InputLayer(dims=dims[1:]),
    layers.Dense(weights=weights, biases=biases, rectifying=True, weight_acc=255),
])
# optional neuron probes
relcos = [quartz.probe(neuron) for neuron in loihi_model.layers[1].output_neurons]
sync = quartz.probe(loihi_model.layers[1].sync_neurons[0])

# build equivalent pyTorch model
pt_model = nn.Sequential(
    nn.Linear(in_features=np.product(dims[1:]), out_features=n_outputs), 
    nn.ReLU()
)
q_weights, q_biases, q_inputs = quartz.utils.quantize_values(weights, biases, inputs, loihi_model.layers[1].weight_acc, t_max)
pt_model[0].weight = torch.nn.Parameter(torch.tensor(q_weights))
pt_model[0].bias = torch.nn.Parameter(torch.tensor((q_biases)))
model_output = pt_model(torch.tensor(q_inputs.reshape(dims[0], -1))).detach().numpy()
model_output

In [ ]:
loihi_model(inputs, partition='loihi')

## Convolutional Layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

t_max = 2**7
input_dims = (1,2,3,3) # (50,10,5,5) # 
weight_dims = (4,2,2,2) # (5,10,5,5) # 
padding = 1
groups = 1
kernel_size = weight_dims[2:]
np.random.seed(seed=27)
weights = (np.random.rand(*weight_dims)-0.5) / 1 # 
# weights = np.ones((weight_dims)) / 4 # 
# weights[1,:,:,:] /= 2
biases = (np.random.rand(weight_dims[0])-0.5) / 2 # 
biases = np.zeros((weight_dims[0])) # 
inputs = np.random.rand(*input_dims) / 3 # 
# inputs = np.ones(input_dims)

loihi_model = quartz.Network(t_max, [
    layers.InputLayer(dims=input_dims[1:]),
    layers.Conv2D(weights=weights, biases=biases, padding=padding, groups=groups, weight_acc=128),
])

relcos = [quartz.probe(neuron) for neuron in loihi_model.layers[1].output_neurons]
sync = quartz.probe(loihi_model.layers[1].sync_neurons[0])
rectifier = quartz.probe(loihi_model.layers[1].rectifier_neurons[0])

q_weights, q_biases, q_inputs = quartz.utils.quantize_values(weights, biases, inputs, loihi_model.layers[1].weight_acc, t_max)
model = nn.Sequential(nn.Conv2d(in_channels=weight_dims[1]*groups, out_channels=weight_dims[0], kernel_size=kernel_size, padding=padding, groups=groups), nn.ReLU())
model[0].weight = torch.nn.Parameter(torch.tensor(q_weights))
model[0].bias = torch.nn.Parameter(torch.tensor((q_biases)))
model_output = model(torch.tensor(q_inputs)).detach().numpy()
model_output

In [ ]:
loihi_output = loihi_model(inputs)
loihi_output

In [ ]:
loihi_model.layers[1].output_neurons

In [ ]:
ok = [relco.plot() for relco in relcos]

In [ ]:
sync.plot()

In [ ]:
rect.plot()

## Maxpool2d layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

t_max = 2**8
input_dims = (1,2,4,4)
weight_dims = (2,2,2,2)
kernel_size = [2,2]

np.random.seed(seed=27)
np.set_printoptions(suppress=True)
weights = (np.random.rand(*weight_dims)-0.5) / 2 # np.zeros((weight_dims)) # 
biases = np.zeros((weight_dims[0]))  # (np.random.rand(weight_dims[0])-0.5) / 3 # 
inputs = np.random.rand(*input_dims) / 2 # np.ones((input_dims)) / 2 #

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims[1:]),
    layers.MaxPool2D(kernel_size=kernel_size),
    layers.Conv2D(weights=weights, biases=biases),
])

wtas = [quartz.probe(block) for block in loihi_model.layers[1].blocks[1:]]
relcos = [quartz.probe(block) for block in loihi_model.layers[2].blocks if 'relco' in block.name]
trigger1 = quartz.probe(loihi_model.layers[1].blocks[0])
trigger2 = quartz.probe(loihi_model.layers[2].blocks[0])

q_weights, q_biases, q_inputs = quartz.utils.quantize_values(weights, biases, inputs, loihi_model.layers[2].weight_acc, t_max)
model = nn.Sequential(
    nn.MaxPool2d(kernel_size=kernel_size),
    nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=weight_dims[2]), 
    nn.ReLU(),
)
model[1].weight = torch.nn.Parameter(torch.tensor(q_weights))
model[1].bias = torch.nn.Parameter(torch.tensor((q_biases)))
model_output = model(torch.tensor(q_inputs)).detach().numpy()
model_output

In [ ]:
loihi_model(inputs, t_max, partition='loihi')

In [ ]:
loihi_model.layers[1].output_blocks()

In [ ]:
trigger1.plot()

In [ ]:
trigger2.plot()

In [ ]:
inputs

In [ ]:
ok = [relco.plot() for relco in relcos]

In [ ]:
ok = [wta.plot() for wta in wtas]

In [ ]:
%debug